<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_video_transcript_translate_with_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id="toc"></a>
# Auto Generated Agent Chat: Translating Video audio using Whisper and GPT-3.5-turbo
In this notebook, we demonstrate how to use whisper and GPT-3.5-turbo with `AssistantAgent` and `UserProxyAgent` to recognize and translate
the speech sound from a video file and add the timestamp like a subtitle file based on [agentchat_function_call.ipynb](https://github.com/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb)


## Requirements
AutoGen requires `Python>=3.8`. To run this notebook example, please install `openai`, `pyautogen`, `whisper`, and `moviepy`:
```bash
pip install openai
pip install openai-whisper
pip install moviepy
pip install pyautogen
```

In [2]:
%%capture --no-stderr
%pip install moviepy~=1.0.3
%pip install openai-whisper~=20231117
%pip install openai~=1.3.5
%pip install pyautogen~=0.2.0b4

ERROR: Invalid requirement: 'openai-whisper~=20230918'


## Set your API Endpoint
It is recommended to store your OpenAI API key in the environment variable. For example, store it in `OPENAI_API_KEY`.

In [4]:
from openai import OpenAI
import whisper
import autogen
from moviepy.editor import VideoFileClip
import os

# print(os.getenv("OPENAI_API_KEY"))

config_list = [
    {
        'model': 'gpt-4',
        'api_key': os.getenv("OPENAI_API_KEY"),
    }
]

print(whisper.available_models())

['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large']


## Example and Output
Below is an example of speech recognition from a [Peppa Pig cartoon video clip](https://drive.google.com/file/d/1QY0naa2acHw2FuH7sY3c-g2sBLtC2Sv4/view?usp=drive_link) originally in English and translated into Chinese.
'FFmpeg' does not support online files. To run the code on the example video, you need to download the example video locally. You can change `your_file_path` to your local video file path.

In [8]:
def recognize_transcript_from_audio(audio_filepath):
    try:
        # client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        # Load model
        model = whisper.load_model("small")

        # Transcribe audio with detailed timestamps
        result = model.transcribe(audio_filepath, verbose=True)

        # Initialize variables for transcript
        transcript = []
        sentence = ""
        start_time = 0

        # Iterate through the segments in the result
        for segment in result['segments']:
            # If new sentence starts, save the previous one and reset variables
            if segment['start'] != start_time and sentence:
                transcript.append({
                    "sentence": sentence.strip() + ".",
                    "timestamp_start": start_time,
                    "timestamp_end": segment['start']
                })
                sentence = ""
                start_time = segment['start']

            # Add the word to the current sentence
            sentence += segment['text'] + " "

        # Add the final sentence
        if sentence:
            transcript.append({
                "sentence": sentence.strip() + ".",
                "timestamp_start": start_time,
                "timestamp_end": result['segments'][-1]['end']
            })

        # Save the transcript to a file
        with open("transcription.txt", "w") as file:
            for item in transcript:
                sentence = item["sentence"]
                start_time, end_time = item["timestamp_start"], item["timestamp_end"]
                file.write(f"{start_time}s to {end_time}s: {sentence}\n")

        return transcript

    except FileNotFoundError:
        return "The specified audio file could not be found."
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"



def translate_text(input_text, source_language, target_language):
    client = OpenAI(api_key=key)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user",
             "content": f"Directly translate the following {source_language} text to a pure {target_language} "
                        f"video subtitle text without additional explanation.: '{input_text}'"},
        ],
        max_tokens=1500
    )

    # Correctly accessing the response content
    translated_text = response.choices[0].message.content if response.choices else None
    return translated_text


def translate_transcript(source_language, target_language):
    with open("transcription.txt", "r") as f:
        lines = f.readlines()

    translated_transcript = []

    for line in lines:
        # Split each line into timestamp and text parts
        parts = line.strip().split(': ')
        if len(parts) == 2:
            timestamp, text = parts[0], parts[1]
            # Translate only the text part
            translated_text = translate_text(text, source_language, target_language)
            # Reconstruct the line with the translated text and the preserved timestamp
            translated_line = f"{timestamp}: {translated_text}"
            translated_transcript.append(translated_line)
        else:
            # If the line doesn't contain a timestamp, add it as is
            translated_transcript.append(line.strip())

    return '\n'.join(translated_transcript)


# llm_config = {
#     "functions": [
#         {
#             "name": "recognize_transcript_from_audio",
#             "description": "recognize the speech from audio and transfer into a txt file",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "audio_filepath": {
#                         "type": "string",
#                         "description": "path of the audio file",
#                     }
#                 },
#                 "required": ["audio_filepath"],
#             },
#         },
#         {
#             "name": "translate_transcript",
#             "description": "using translate_text function to translate the script",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "source_language": {
#                         "type": "string",
#                         "description": "source language",
#                     },
#                     "target_language": {
#                         "type": "string",
#                         "description": "target language",
#                     }
#                 },
#                 "required": ["source_language", "target_language"],
#             },
#         },
#     ],
#     "config_list": config_list,
#     "timeout": 120,
# }
source_language = "Spanish"
target_language = "English"
key = os.getenv("OPENAI_API_KEY")
target_video = "audio.mp3"

recognize_transcript_from_audio(target_video)

# chatbot = autogen.AssistantAgent(
#     name="chatbot",
#     system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
#     llm_config=llm_config,
# )

# user_proxy = autogen.UserProxyAgent(
#     name="user_proxy",
#     is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=10,
#     code_execution_config={"work_dir": "coding_2"},
# )

# user_proxy.register_function(
#     function_map={
#         "recognize_transcript_from_audio": recognize_transcript_from_audio,
#         "translate_transcript": translate_transcript,
#     }
# )
# user_proxy.initiate_chat(
#     chatbot,
#     message=f"For the audio located in {target_video}, recognize the speech and transfer it into a script file, "
#             f"then translate from {source_language} text to a {target_language} video subtitle text. ",
# )

100%|███████████████████████████████████████| 461M/461M [01:08<00:00, 7.10MiB/s]


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Spanish
[00:00.000 --> 00:08.980]  verde, una versión así moderna que dice así comenzando con el versículo 16 del capítulo
[00:08.980 --> 00:22.720]  28 de Mateo, entonces los once discípulos salieron hacia Galilea y se dijeron al monte
[00:22.720 --> 00:31.920]  que Jesús le había indicado cuando vieron a Jesús lo adoraron pero algunos de ellos
[00:31.920 --> 00:44.640]  ayudaban. Jesús se acercó y dijo a sus discípulos, se me ha dado toda autoridad en el cielo y en la
[00:44.640 --> 00:56.660]  tierra por lo tanto vayan y hagan discípulos de todas las naciones pautizándolos en el nombre del
[00:56.660 --> 01:10.380]  padre del hijo del Espíritu Santo. Enseñen a los discípulos a obedecer todos los mandatos que
[01:10.380 --> 01:21.660]  les he dado y tengan por seguro esto que estoy con ustedes siempre hasta el fin de los tiempos.
[01:21.660 --> 01:31.140]  Entonces con est

[{'sentence': 'verde, una versión así moderna que dice así comenzando con el versículo 16 del capítulo.',
  'timestamp_start': 0,
  'timestamp_end': 8.98},
 {'sentence': '28 de Mateo, entonces los once discípulos salieron hacia Galilea y se dijeron al monte.',
  'timestamp_start': 8.98,
  'timestamp_end': 22.72},
 {'sentence': 'que Jesús le había indicado cuando vieron a Jesús lo adoraron pero algunos de ellos.',
  'timestamp_start': 22.72,
  'timestamp_end': 31.92},
 {'sentence': 'ayudaban. Jesús se acercó y dijo a sus discípulos, se me ha dado toda autoridad en el cielo y en la.',
  'timestamp_start': 31.92,
  'timestamp_end': 44.64},
 {'sentence': 'tierra por lo tanto vayan y hagan discípulos de todas las naciones pautizándolos en el nombre del.',
  'timestamp_start': 44.64,
  'timestamp_end': 56.66},
 {'sentence': 'padre del hijo del Espíritu Santo. Enseñen a los discípulos a obedecer todos los mandatos que.',
  'timestamp_start': 56.66,
  'timestamp_end': 70.38},
 {'sentence': 'le